In [6]:

import pandas
from keras.models import Sequential
from keras.layers import Dense

# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer

In [7]:

dataframe = pandas.read_csv("persistence/training.csv")
dataset = dataframe.drop('awardedMatches', axis=1).drop('id', axis=1).values
X = dataset[:,0:98].astype(float)
y = dataset[:, 98:].astype(int)
st_x= Normalizer()
X= st_x.fit_transform(X)
print(X.shape, y.shape)

(65, 98) (65, 3)


In [8]:

# define the model
model = Sequential()
model.add(Dense(20, input_dim=98, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
7/7 [==============================] - 1s 3ms/step - loss: 0.6911
Epoch 2/150
7/7 [==============================] - 0s 3ms/step - loss: 0.6827
Epoch 3/150
7/7 [==============================] - 0s 3ms/step - loss: 0.6754
Epoch 4/150
7/7 [==============================] - 0s 3ms/step - loss: 0.6686
Epoch 5/150
7/7 [==============================] - 0s 3ms/step - loss: 0.6628
Epoch 6/150
7/7 [==============================] - 0s 5ms/step - loss: 0.6574
Epoch 7/150
7/7 [==============================] - 0s 3ms/step - loss: 0.6527
Epoch 8/150
7/7 [==============================] - 0s 3

In [9]:

...
results = list()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# define evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# enumerate folds
for train_ix, test_ix in cv.split(X):
	# prepare data
	X_train, X_test = X[train_ix], X[test_ix]
	y_train, y_test = y[train_ix], y[test_ix]
	# fit model
	model.fit(X_train, y_train, verbose=0, epochs=100)
	# make a prediction on the test set
	yhat = model.predict(X_test)
	# round probabilities to class labels
	yhat = yhat.round()
	# calculate accuracy
	acc = accuracy_score(y_test, yhat)
	# store result
	print('>%.2f' % acc)
	results.append(acc)

>0.29
>0.57
>0.86
>0.43
>0.43
>1.00
>0.67
>1.00
>0.33
>0.67
>0.57
>0.71
>1.00
>0.86
>0.86
>0.83
>1.00
>0.83
>1.00
>0.83
>0.86
>0.86
>1.00
>1.00
>1.00
>1.00
>1.00
>0.83
>1.00
>1.00


In [10]:
print('Average accuracy: %.2f%% \nStandart deviation: %.2f%%' % ((mean(results))*100, (std(results)*100)))

Average accuracy: 80.95% 
Standart deviation: 21.63%
